In [23]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [24]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm
import random
import json

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id']

cate_cols = ['store_id_encoded', 'area_name', 'city', 'genre_name']
cate_feats = ['dow', 'hol_days', 'day', 'pom', 'prev_is_holiday', 'next_is_holiday', 
              'wom', 'woy', 'is_weekends', 'holiday_flg', 'month', 'is_up_corner']
for mod in ['air', 'hpg']:
    cate_feats.extend(['%s_%s' % (mod, c) for c in cate_cols])
    
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
kfold = 5
strategy = 'lgb_l2'
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    #
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
##### model selection with CV
# score
holdout_score = .0
# parameters
params = {
    "boosting": "gbdt",
    "objective": "regression_l2",
    "lambda_l2": [200, 20, 1],
    "learning_rate": [0.4, 0.2, 0.1],
    "max_depth": [8, 12, 16],
    
#     "lambda_l2": [20],
#     "learning_rate": [0.2],
#     "max_depth": [16],
    
    "num_iterations": 150,
    "min_data_in_leaf": 20,
    'num_leaves': 255,

    "feature_fraction": 0.90,
    "bagging_fraction": 0.85,
    "bagging_freq": 20,
    "min_hessian": 0.001,

    "max_bin": 63,
}
total_cols = [c for c in TrainData.columns if c not in drop_cols]
K = int(0.3 * len(total_cols))
N = 100

start = time.time()
for idx in range(N): 
    selected_cols = [total_cols[i] for i in sorted(random.sample(range(len(total_cols)), K))]
    OutputDir = '%s/MM/l1/%s' % (DataBaseDir, idx)
    if(os.path.exists(OutputDir) == False):
        os.makedirs(OutputDir)
    # save feature space
    with open('%s/sub_feats.txt' % OutputDir, 'w') as o_file:
        for feat in selected_cols:
            o_file.write('%s\n' % feat)
    o_file.close()
    ##
    BestParmas = {}
    BestScore = 1.0
    cv_score = .0
    for l2 in params['lambda_l2']:
        for lr in params['learning_rate']:
            for depth in params['max_depth']:
                cv_rmsle = .0
                for fold in range(kfold):
                    FoldData = {
                        'train': TrainData[TrainData['fold'] != fold],
                        'valid': TrainData[TrainData['fold'] == fold]
                    }
                    d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                                        label= FoldData['train']['visitors'], 
                                        #max_bin= params['max_bin'], 
                                        silent= True, 
                                        free_raw_data= True)
                    param = {
                        'boosting': 'gbdt',
                        'objective': 'regression_l2',

                        'lambda_l2': l2,
                        'learning_rate': lr,
                        'max_depth': depth,

                        'num_iterations': params['num_iterations'],
                        'feature_fraction': params['feature_fraction'],
                        'bagging_fraction': params['bagging_fraction'],
                        'bagging_freq': params['bagging_freq'],
                        'min_hessian': params['min_hessian'],
                        'max_bin': params['max_bin'],
                    }
                    model = lightgbm.train(param, d_cv)
                    FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
                    rmsle_valid = RMSLE(FoldData['valid']['visitors'], FoldData['valid'][strategy])
                    cv_rmsle += rmsle_valid
                cv_rmsle /= kfold
                if(cv_rmsle < BestScore):
                    BestScore = cv_rmsle
                    BestParmas['lambda_l2'] = l2
                    BestParmas['learning_rate'] = lr
                    BestParmas['max_depth'] = depth
                end = time.time()
                print('running for params, l2 %s, lr %s, depth %s done, cv score %.5f, time elapsed %.2fs' % (l2, lr, depth, cv_rmsle, (end - start)))
    end = time.time()
    print('grid search done, time elapsed %.2fs' % (end - start))
    print('best params, %s' % BestParmas)
    ## retrain and store
    cv_score = .0
    holdout_score = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold],
            'holdout': holdout_dfs[fold],
            'test': test_dfs[fold]
        }
        # train
        d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                            label= FoldData['train']['visitors'].values, 
                            #max_bin= params['max_bin'], 
                            silent= True, 
                            free_raw_data= True)
        param = {
            'boosting': 'gbdt',
            'objective': 'regression_l2',
                
            'lambda_l2': BestParmas['lambda_l2'],
            'learning_rate': BestParmas['learning_rate'],
            'max_depth': BestParmas['max_depth'],
                        
            'num_iterations': params['num_iterations'],
            'feature_fraction': params['feature_fraction'],
            'bagging_fraction': params['bagging_fraction'],
            'bagging_freq': params['bagging_freq'],
            'min_hessian': params['min_hessian'],
            'max_bin': params['max_bin'],
        }
        model = lightgbm.train(param, d_cv)
        # for valid
        FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
        cv_score += rmsle_valid
        # for holdout
        FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][selected_cols])
        rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
        holdout_score += rmsle_holdout
        # for test
        FoldData['test'][strategy] = model.predict(FoldData['test'][selected_cols])

        print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
        #### output
        FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        for mod in FoldData.keys():
            if(mod == 'train'):
                continue
            OutCols = []
            if(mod == 'test'):
                OutCols.append('id')
            OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
            OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
            OutFoldData = FoldData[mod][OutCols]
            OutFoldData.to_csv(OutputFile, index= False)
    
    cv_score /= kfold # Average valid set predictions
    holdout_score /= kfold # Average holdout set predictions

    end = time.time()
    print('\n======================')
    print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, (end - start)))
    print('======================\n')
    
    with open('%s/result.txt' % OutputDir, 'w') as o_file:
        o_file.write('%s\n' % json.dumps(BestParmas))
        o_file.write('cv score %.4f, holdout score %.4f' % (cv_score, holdout_score))
    o_file.close()
    
#     if(idx == 0):
#         break

/home/joe/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


running for params, l2 200, lr 0.4, depth 8 done, cv score 0.51253, time elapsed 3.95s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.51249, time elapsed 7.72s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.51234, time elapsed 11.52s
running for params, l2 200, lr 0.2, depth 8 done, cv score 0.51317, time elapsed 15.58s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.51296, time elapsed 19.53s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.51287, time elapsed 23.49s
running for params, l2 200, lr 0.1, depth 8 done, cv score 0.51511, time elapsed 27.83s
running for params, l2 200, lr 0.1, depth 12 done, cv score 0.51496, time elapsed 32.21s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.51490, time elapsed 36.56s
running for params, l2 20, lr 0.4, depth 8 done, cv score 0.51282, time elapsed 39.95s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.51267, time elapsed 43.38s
running for params, l2 20, lr

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.516249, holdout score 0.506520, valid length 45371
fold 1: valid score 0.507739, holdout score 0.507100, valid length 45371
fold 2: valid score 0.511945, holdout score 0.507308, valid length 45371
fold 3: valid score 0.515262, holdout score 0.507209, valid length 45370
fold 4: valid score 0.510486, holdout score 0.507075, valid length 45370

CV score 0.5123, Holdout score 0.5070, Elapsed time: 109.92s

running for params, l2 200, lr 0.4, depth 8 done, cv score 0.53832, time elapsed 113.15s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.53801, time elapsed 116.34s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.53777, time elapsed 119.54s
running for params, l2 200, lr 0.2, depth 8 done, cv score 0.54026, time elapsed 122.92s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.53977, time elapsed 126.36s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.53972, time elapsed 129.62s
running for params, l2 200, lr 0.1

running for params, l2 1, lr 0.4, depth 16 done, cv score 0.52076, time elapsed 378.39s
running for params, l2 1, lr 0.2, depth 8 done, cv score 0.51742, time elapsed 381.51s
running for params, l2 1, lr 0.2, depth 12 done, cv score 0.51722, time elapsed 384.62s
running for params, l2 1, lr 0.2, depth 16 done, cv score 0.51747, time elapsed 387.71s
running for params, l2 1, lr 0.1, depth 8 done, cv score 0.51804, time elapsed 391.17s
running for params, l2 1, lr 0.1, depth 12 done, cv score 0.51793, time elapsed 394.64s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.51797, time elapsed 398.10s
grid search done, time elapsed 398.10s
best params, {'lambda_l2': 20, 'learning_rate': 0.2, 'max_depth': 12}
fold 0: valid score 0.519785, holdout score 0.512004, valid length 45371
fold 1: valid score 0.512834, holdout score 0.511696, valid length 45371
fold 2: valid score 0.516694, holdout score 0.510895, valid length 45371
fold 3: valid score 0.521332, holdout score 0.511436, vali

...

running for params, l2 20, lr 0.1, depth 16 done, cv score 0.50467, time elapsed 8663.55s
running for params, l2 1, lr 0.4, depth 8 done, cv score 0.50523, time elapsed 8666.48s
running for params, l2 1, lr 0.4, depth 12 done, cv score 0.50529, time elapsed 8669.38s
running for params, l2 1, lr 0.4, depth 16 done, cv score 0.50532, time elapsed 8672.28s
running for params, l2 1, lr 0.2, depth 8 done, cv score 0.50350, time elapsed 8675.43s
running for params, l2 1, lr 0.2, depth 12 done, cv score 0.50298, time elapsed 8678.58s
running for params, l2 1, lr 0.2, depth 16 done, cv score 0.50328, time elapsed 8681.72s
running for params, l2 1, lr 0.1, depth 8 done, cv score 0.50467, time elapsed 8685.30s
running for params, l2 1, lr 0.1, depth 12 done, cv score 0.50447, time elapsed 8688.85s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.50443, time elapsed 8692.40s
grid search done, time elapsed 8692.40s
best params, {'lambda_l2': 20, 'learning_rate': 0.2, 'max_depth': 12}
fo

running for params, l2 200, lr 0.1, depth 16 done, cv score 0.60691, time elapsed 8921.37s
running for params, l2 20, lr 0.4, depth 8 done, cv score 0.59478, time elapsed 8924.81s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.59351, time elapsed 8928.26s
running for params, l2 20, lr 0.4, depth 16 done, cv score 0.59352, time elapsed 8931.76s
running for params, l2 20, lr 0.2, depth 8 done, cv score 0.59835, time elapsed 8935.48s
running for params, l2 20, lr 0.2, depth 12 done, cv score 0.59757, time elapsed 8939.19s
running for params, l2 20, lr 0.2, depth 16 done, cv score 0.59798, time elapsed 8942.91s
running for params, l2 20, lr 0.1, depth 8 done, cv score 0.60602, time elapsed 8947.08s
running for params, l2 20, lr 0.1, depth 12 done, cv score 0.60487, time elapsed 8951.31s
running for params, l2 20, lr 0.1, depth 16 done, cv score 0.60443, time elapsed 8955.49s
running for params, l2 1, lr 0.4, depth 8 done, cv score 0.59536, time elapsed 8958.88s
running for pa

running for params, l2 200, lr 0.4, depth 8 done, cv score 0.55211, time elapsed 9187.65s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.55144, time elapsed 9191.30s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.55203, time elapsed 9194.94s
running for params, l2 200, lr 0.2, depth 8 done, cv score 0.55661, time elapsed 9198.75s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.55565, time elapsed 9202.65s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.55545, time elapsed 9206.50s
running for params, l2 200, lr 0.1, depth 8 done, cv score 0.56126, time elapsed 9210.70s
running for params, l2 200, lr 0.1, depth 12 done, cv score 0.56045, time elapsed 9215.04s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.56023, time elapsed 9219.40s
running for params, l2 20, lr 0.4, depth 8 done, cv score 0.55125, time elapsed 9222.76s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.55047, time elapsed 9226.09s
runni

running for params, l2 1, lr 0.1, depth 12 done, cv score 0.51679, time elapsed 9467.59s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.51693, time elapsed 9471.15s
grid search done, time elapsed 9471.15s
best params, {'lambda_l2': 200, 'learning_rate': 0.4, 'max_depth': 16}
fold 0: valid score 0.518215, holdout score 0.511341, valid length 45371
fold 1: valid score 0.511979, holdout score 0.511954, valid length 45371
fold 2: valid score 0.515872, holdout score 0.511086, valid length 45371
fold 3: valid score 0.519849, holdout score 0.510551, valid length 45370
fold 4: valid score 0.514114, holdout score 0.510878, valid length 45370

CV score 0.5160, Holdout score 0.5112, Elapsed time: 9476.67s

running for params, l2 200, lr 0.4, depth 8 done, cv score 0.54983, time elapsed 9479.94s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.54911, time elapsed 9483.25s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.54934, time elapsed 9486.50s
running fo

running for params, l2 20, lr 0.1, depth 12 done, cv score 0.52084, time elapsed 9720.81s
running for params, l2 20, lr 0.1, depth 16 done, cv score 0.52090, time elapsed 9724.24s
running for params, l2 1, lr 0.4, depth 8 done, cv score 0.52383, time elapsed 9727.08s
running for params, l2 1, lr 0.4, depth 12 done, cv score 0.52375, time elapsed 9729.91s
running for params, l2 1, lr 0.4, depth 16 done, cv score 0.52425, time elapsed 9732.72s
running for params, l2 1, lr 0.2, depth 8 done, cv score 0.52070, time elapsed 9735.75s
running for params, l2 1, lr 0.2, depth 12 done, cv score 0.52090, time elapsed 9738.73s
running for params, l2 1, lr 0.2, depth 16 done, cv score 0.52084, time elapsed 9741.73s
running for params, l2 1, lr 0.1, depth 8 done, cv score 0.52094, time elapsed 9745.08s
running for params, l2 1, lr 0.1, depth 12 done, cv score 0.52086, time elapsed 9748.42s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.52089, time elapsed 9751.75s
grid search done, time